# Getting started quickly with Gate Set Tomography

The `pygsti` package provides multiple levels of abstraction over the core Gate Set Tomography (GST) algorithms.  This initial tutorial will show you how to work with `pygsti`'s highest level of abstraction to get you started using GST quickly.  Subsequent tutorials will delve into the details of `pygsti` objects and algorithms, and how to use them in detail.

## The `do_long_sequence_gst` driver function
Let's first look at how to use the  `do_long_sequence_gst` which combines all the steps of running typical GST algortithms into a single function. 

In [1]:
#Make print statements compatible with Python 2 and 3
from __future__ import print_function

#Set matplotlib backend: the ipython "inline" backend cannot pickle
# figures, which is required for generating pyGSTi reports
import matplotlib  
matplotlib.use('Agg')

In [2]:
#Import the pygsti module (always do this)
import pygsti

#### First, we need to specify what our desired gate set is, referred to as the "target gateset".
Gate sets and other `pygsti` objects are constructed using routines within `pygsti.construction`, and so we construct a gateset by calling `pygsti.construction.build_gateset`:

In [3]:
#Construct a target gateset
gs_target = pygsti.construction.build_gateset([2], [('Q0',)], ['Gi', 'Gx', 'Gy'],
                                              ["I(Q0)", "X(pi/2,Q0)", "Y(pi/2,Q0)"],
                                              prepLabels=['rho0'], prepExpressions=["0"],
                                              effectLabels=['E0'], effectExpressions=["1"],
                                              spamdefs={'plus': ('rho0', 'E0'),
                                                        'minus': ('rho0', 'remainder')})

The parameters to `build_gateset`, specify:
 - The state space is dimension 2 (i.e. the density matrix is 2x2)
 
 - interpret this 2-dimensional space as that of a single qubit labeled "Q0" (label must begin with 'Q')
 
 - there are three gates: Idle, $\pi/2$ x-rotation, $\pi/2$ y-rotation, labeled `Gi`, `Gx`, and `Gy`.
 
 - there is one state prep operation, labeled `rho0`, which prepares the 0-state (the first basis element of the 2D state space)
 
 - there is one POVM (~ measurement), labeled `E0` that projects onto the 1-state (the second basis element of the 2D state space)
 
 - the name of the state-prep then measure our POVM is `plus`
 
 - the name of the state-prep then measure something other than our POVM is `minus` 

Reading from and writing to files is done mostly via routines in `pygsti.io`. To store this gateset in a file (for reference or to load it somewhere else), you just call `pygsti.io.write_gateset`:

In [4]:
#Write it to a file
pygsti.io.write_gateset(gs_target, "tutorial_files/MyTargetGateset.txt")

#To load the gateset back into a python object, do:
# gs_target = pygsti.io.load_gateset("tutorial_files/MyTargetGateset.txt")

#### Next, we need to create fiducial, germ, and max-length lists:
These three lists will specify what experiments GST will use in its estimation procedure, and depend on the target gateset as well as the expected quality of the qubit being measured.  They are:

- fiducial gate strings (``fiducials``): gate sequences that immediately follow state preparation or immediately precede measurement.


- germ gate strings (``germs``): gate sequences that are repeated to produce a string that is as close to some "maximum length" as possible without exceeding it.


- maximum lengths (`maxLengths`): a list of maximum lengths used to specify the increasingly long gate sequences (via more germ repeats) used by each iteration of the GST estimation procedure.

To make GST most effective, these gate strings lists should be computed.  Typically this computation is done by the Sandia GST folks and the gate string lists are sent to you, though there is preliminary support within `pygsti` for computing these string lists directly.  Here, we'll assume we have been given the lists.  The maximum lengths list typically starts with [0,1] and then contains successive powers of two.  The largest maximum length should roughly correspond to the number of gates ones qubit can perform before becoming depolarized beyond ones ability to measure anything other than the maximally mixed state.  Since we're constructing gate string lists, the routines used are in `pygsti.construction`:

In [5]:
#Create fiducial gate string lists
fiducials = pygsti.construction.gatestring_list( [ (), ('Gx',), ('Gy',), ('Gx','Gx'), ('Gx','Gx','Gx'), ('Gy','Gy','Gy') ])

#Create germ gate string lists
germs = pygsti.construction.gatestring_list( [('Gx',), ('Gy',), ('Gi',), ('Gx', 'Gy',),
         ('Gx', 'Gy', 'Gi',), ('Gx', 'Gi', 'Gy',), ('Gx', 'Gi', 'Gi',), ('Gy', 'Gi', 'Gi',),
         ('Gx', 'Gx', 'Gi', 'Gy',), ('Gx', 'Gy', 'Gy', 'Gi',),
         ('Gx', 'Gx', 'Gy', 'Gx', 'Gy', 'Gy',)] )

#Create maximum lengths list
maxLengths = [1,2,4,8,16,32]

If we want to, we can save these lists in files (but this is not necessary):

In [6]:
pygsti.io.write_gatestring_list("tutorial_files/MyFiducials.txt", fiducials, "My fiducial gate strings")
pygsti.io.write_gatestring_list("tutorial_files/MyGerms.txt", germs, "My germ gate strings")

import pickle
pickle.dump( maxLengths, open("tutorial_files/MyMaxLengths.pkl", "wb"))

# To load these back into python lists, do:
#fiducials = pygsti.io.load_gatestring_list("tutorial_files/MyFiducials.txt")
#germs = pygsti.io.load_gatestring_list("tutorial_files/MyGerms.txt")
#maxLengths = pickle.load( open("tutorial_files/MyMaxLengths.pkl"))

#### Third, we generate (since we can't actually take) data and save a dataset
Before experimental data is obtained, it is useful to create a "template" dataset file which specifies which gate sequences are required to run GST.  Since we don't actually have an experiment for this example, we'll generate some "fake" experimental data from a set of gates that are just depolarized versions of the targets.  First we construct the list of experiments used by GST using `make_lsgst_experiment_list`, and use the result to specify which experiments to simulate.  The abbreviation "LSGST" (lowercase in function names to follow Python naming conventions) stands for "Long Sequence Gate Set Tomography", and refers to the more powerful flavor of GST that utilizes long sequences to find gate set estimates.  LSGST can be compared to Linear GST, or "LGST", which only uses short sequences and as a result provides much less accurate estimates.

In [7]:
#Create a list of GST experiments for this gateset, with
#the specified fiducials, germs, and maximum lengths
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(
    gs_target, fiducials, fiducials, germs, maxLengths)

#Create an empty dataset file, which stores the list of experiments
#plus extra columns where data can be inserted
pygsti.io.write_empty_dataset("tutorial_files/MyDataTemplate.txt", listOfExperiments,
                              "## Columns = plus count, count total")

Since we don't actually have a experiment to generate real data, let's now create and save a dataset using depolarized target gates and spam operations:

In [8]:
#Create a gateset of depolarized gates and SPAM relative to target, and generate fake data using this gateset.
gs_datagen = gs_target.depolarize(gate_noise=0.1, spam_noise=0.001)
ds = pygsti.construction.generate_fake_data(gs_datagen, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=2015)


We could at this point just use the generated dataset directly, but let's save it as though it were a file filled with experimental results.

In [9]:
#Save our dataset
pygsti.io.write_dataset("tutorial_files/MyDataset.txt", ds)

#Note; to load the dataset back again, do:
#ds = pygsti.io.load_dataset("tutorial_files/MyDataset.txt")

#### (OUTDATED!!!) Fourth, we call the Analysis function
Now we're all set to call the driver routine.  All of the possible arguments to this function are detailed in the included help (docstring), and so here we just make a few remarks:
- For many of the arguments, you can supply either a filename or a python object (e.g. dataset, target gateset, gate string lists).


- `fiducials` is supplied twice since the state preparation fiducials (those sequences following a state prep) need not be the same as the measurement fiducials (those sequences preceding a measurement).


- Typically we want to constrain the resulting gates to be trace-preserving.  This is accomplished by the call to `set_all_parameterizations("TP")`, which restricts `gs_target` to only describing TP gates (see more on this in later tutorials).


- `gaugeOptParams` specifies a dictionary of parameters ultimately to be passed to the `gaugeopt_to_target` function (which provides full documentation).  We set the ratio of the state preparation and measurement (SPAM) weighting to the gate weighting when performing a gauge optimization.  When this is set as below, the gate parameters are weighted 1000 times more relative to the SPAM parameters.  Mathematically this corresponds to a multiplicative factor of 0.001 preceding the sum-of-squared-difference terms corresponding to SPAM elements in the gateset.   Typically it is good to weight the gates parameters more heavily since GST amplifies gate parameter errors via long gate sequences but cannot amplify SPAM parameter errors.  If unsure, 0.001 is a good value to start with.

In [10]:
results = pygsti.do_stdpractice_gst("tutorial_files/MyDataset.txt", gs_target,                                     
                                    fiducials, fiducials, germs, maxLengths)


Loading tutorial_files/MyDataset.txt: 100%
Writing cache file (to speed future loads): tutorial_files/MyDataset.txt.cache
-- Std Practice:  Iter 1 of 2  (TP) --: 
  --- LGST ---
  --- Iterative MLGST: [##################################################] 100.0%  1585 gate strings ---
  Iterative MLGST Total Time: 1.6s
-- Std Practice:  Iter 2 of 2  (CPTP) --: 
  --- Iterative MLGST: [##################################################] 100.0%  1585 gate strings ---
  Iterative MLGST Total Time: 3.1s


In [18]:
import pickle
s = pickle.dumps(results)
r2 = pickle.loads(s)
print(r2.estimates['TP'].gatesets['Spam 0.0001'])


rho0 =    0.7071  -0.0008  -0.0058   0.7050


E0 =    0.7075  -0.0020  -0.0054  -0.7065


Gi = 
   1.0000        0        0        0
  -0.0001   0.9007  -0.0009   0.0026
   0.0003   0.0017   0.8984  -0.0018
  -0.0009  -0.0008   0.0005   0.8950


Gx = 
   1.0000        0        0        0
  -0.0003   0.8985  -0.0009  -0.0002
  -0.0004   0.0021  -0.0032  -0.9019
  -0.0003  -0.0028   0.9019   0.0047


Gy = 
   1.0000        0        0        0
  -0.0003  -0.0001  -0.0046   0.9022
   0.0006  -0.0016   0.8978  -0.0065
   0.0004  -0.9022   0.0035   0.0006





The analysis routine returns a `pygsti.report.Results` object which encapsulates intermediate and final GST estimates, as well as some quantities derived from these "raw" estimates.  These objects can be used when generating HTML reports, using functions within `pygsti.report`, as we demonstrate below. 

In [20]:
# Access to raw GST best gateset estimate (after gauge optimization)
print(results.estimates['TP'].gatesets['Spam 0.0001'])


rho0 =    0.7071  -0.0008  -0.0058   0.7050


E0 =    0.7075  -0.0020  -0.0054  -0.7065


Gi = 
   1.0000        0        0        0
  -0.0001   0.9007  -0.0009   0.0026
   0.0003   0.0017   0.8984  -0.0018
  -0.0009  -0.0008   0.0005   0.8950


Gx = 
   1.0000        0        0        0
  -0.0003   0.8985  -0.0009  -0.0002
  -0.0004   0.0021  -0.0032  -0.9019
  -0.0003  -0.0028   0.9019   0.0047


Gy = 
   1.0000        0        0        0
  -0.0003  -0.0001  -0.0046   0.9022
   0.0006  -0.0016   0.8978  -0.0065
   0.0004  -0.9022   0.0035   0.0006





In [21]:
#create a "full" single-qubit GST report (most detailed and pedagogical; best for those getting familiar with GST)
pygsti.report.create_single_qubit_report(results, confidenceLevel=95, 
                                         filename="tutorial_files/easy_single.html",
                                         verbosity=2, auto_open=True)

*** Generating tables ***
    
--- Hessian Projector Optimization for gate CIs (L-BFGS-B) ---
    9s           0.0278812438
   14s           0.0276211460
   19s           0.0275954134
   24s           0.0275786283
   29s           0.0275733244
   35s           0.0275724603
  The resulting min sqrt(sum(gateCIs**2)): 0.0275725
    
--- Hessian Projector Optimization for gate CIs (L-BFGS-B) ---
   14s           0.1577171383
   19s           0.1576439456
   24s           0.1574537048
   29s           0.1571662062
   33s           0.1568346774
   38s           0.1564345169
   43s           0.1547837952
   53s           0.1533902678
   62s           0.1521120252
   67s           0.1495010380
   72s           0.1359355342
   77s           0.1216415256
   91s           0.1179970796
   96s           0.1151106579
  101s           0.1127600985
  106s           0.1084313952
  111s           0.1058887481
  116s           0.1012151869
  120s           0.0984673271
  125s           0.0970586335
  130

In [22]:
#create a "brief-mode" report, which highlights main results; for those familiar with the report's tables and figures)
pygsti.report.create_single_qubit_report(results, confidenceLevel=95, 
                                         filename="tutorial_files/easy_single_brief.html",
                                         verbosity=2, brief=True, auto_open=True)

*** Generating tables ***
*** Generating plots ***
*** Merging into template file ***
Output written to tutorial_files/easy_single_brief.html
Opening tutorial_files/easy_single_brief.html...


In [23]:
#Creates a report whose format is not specific to single qubits, and which is indended to be
# used for "general" (any) qubit number.
pygsti.report.create_general_report(results, confidenceLevel=95,
                                    filename="tutorial_files/easy_general.html",
                                    verbosity=2, auto_open=True)

*** Generating tables ***
*** Generating plots ***
*** Merging into template file ***
Output written to tutorial_files/easy_general.html
Opening tutorial_files/easy_general.html...


The above lines should have opened each created report in a new tab of your browser. For reference, here are brief descriptions of the types of reports pyGSTi is capable of generating, along with links to the examples we just created.  Each of these reports can be generated in a "full" (`brief=False`) and "brief" (`brief=True`) modes.  The former is much more detailed and pedagogical, and is best for those getting familiar with GST.  The latter mode is useful for veteran users who are already familiar with what the tables and figures in the report signify. 

- **Single-qubit reports** specialize in displaying the results of single-qubit GST.     [tutorial_files/easy_single.html](tutorial_files/easy_single.html) is an example of a full-mode report, and [tutorial_files/easy_single_brief.html](tutorial_files/easy_single_brief.html) a brief-mode report.

- **"General" reports** are intended to display the results of GST on one *or more* qubits (or larger-than-single-qubit Hilbert spaces in general).  [tutorial_files/easy_general.html](tutorial_files/easy_general.html) is an example.  While this report analyzes just a single qubit, the format is not 1-qubit specific.

## Speeding things up by using Standard constructions
A significant component of running GST as show above is constructing things: the target gateset, the fiducial, germ, and maximum-length lists, etc.  We've found that many people who use GST have one of only a few different target gatesets, and for these commonly used target gatesets we've created modules that perform most of the constructions for you.  If you gateset isn't one of these standard ones then you'll have to follow the above approach for now, but please let us know and we'll try to add a module for your gateset in the future.

The standard construction modules are located under `pygsti.construction` (surprise, surprise) and are prefixed with "`std`".  In the example above, our gateset (comprised of single qubit $I$, X($\pi/2$), and Y($\pi/2$) gates) is one of the commonly used gatesets, and relevant constructions are importable via:

In [24]:
#Import the "stardard 1-qubit quantities for a gateset with X(pi/2), Y(pi/2), and idle gates"
from pygsti.construction import std1Q_XYI

We follow the same order of constructing things as above, but it's much easier since almost everything has been constructed already:

In [25]:
gs_target = std1Q_XYI.gs_target
fiducials = std1Q_XYI.fiducials
germs = std1Q_XYI.germs
maxLengths = [1,2,4,8,16,32] #still need to define this manually

We generate a fake dataset as before:

In [26]:
gs_datagen = gs_target.depolarize(gate_noise=0.1, spam_noise=0.001)
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(gs_target.gates.keys(), fiducials, fiducials, germs, maxLengths)
ds = pygsti.construction.generate_fake_data(gs_datagen, listOfExperiments, nSamples=1000000,
                                            sampleError="binomial", seed=1234)

And run the analysis function (this time using the dataset object directly instead of loading from a file), and then create a report in the specified file.

In [27]:
gs_target.set_all_parameterizations("TP")
results = pygsti.do_long_sequence_gst(ds, gs_target, fiducials, fiducials, germs, maxLengths)
pygsti.report.create_single_qubit_report(results, confidenceLevel=95, 
                                         filename="tutorial_files/MyEvenEasierReport.html",
                                         verbosity=2)

--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.24409810599
  1.16704609919
  0.946915237
  0.943164947695
  0.00230163583615
  0.000699324004315
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.24264068712
  1.41421356237
  1.41421356237
  1.41421356237
  3.33818959205e-16
  1.55012476151e-16
  
--- Iterative MLGST: Iter 1 of 6  92 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
   groups of ~1 procs each, to distribute over 43 params (taken as 1 param groups of ~43 params).
  Sum of Chi^2 = 65.4588 (92 data params - 31 model params = expected mean of 61; p-value = 0.32481)
  Completed in 0.1s
  2*Delta(log(L)) = 65.4654
  Iteration 1 took 0.1s
  
--- Iterative MLGST: Iter 2 of 6  168 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
   groups of ~1 pro

*** Generating tables ***
    
--- Hessian Projector Optimization for gate CIs (L-BFGS-B) ---
    9s           0.0008805845
   14s           0.0008721035
   19s           0.0008712885
  The resulting min sqrt(sum(gateCIs**2)): 0.000871289
*** Generating plots ***
*** Merging into template file ***
Output written to tutorial_files/MyEvenEasierReport.html


Now open [tutorial_files/MyEvenEasierReport.html](tutorial_files/MyEvenEasierReport.html) to see the results.  (Note that the report didn't open automatically this time because `auto_open` defaults to `False`.)  You've just run GST (again)!

In [28]:
# Printing a Results object gives you information about how to extract information from it
print(results)

----------------------------------------------------------
---------------- pyGSTi Results Object -------------------
----------------------------------------------------------

How to access my contents:

 .dataset    -- the DataSet used to generate these results

 .gatestring_lists   -- a dict of GateString lists w/keys:
 ---------------------------------------------------------
  iteration
  final
  all
  iteration delta
  prep fiducials
  effect fiducials
  germs

 .gatestring_structs   -- a dict of GatestringStructures w/keys:
 ---------------------------------------------------------
  iteration
  final

 .estimates   -- a dictionary of Estimate objects:
 ---------------------------------------------------------
  default


